# Web安全之机器学习
## 第7章朴素贝叶斯算法
### 7.4 示例：检测WebShell（一）

#### 1. 数据搜集和数据清洗
我们将搜集到的WebShell作为黑样本，wordpress源码作为白样本。将一个PHP文件作为一个字符串处理，基于2-gram切割，形成基于2-gram的词汇表。
（1）将WebShell的每一个样本文件，转化为一个字符串

In [1]:
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

In [2]:
def get_one_file(filename):
    string = ""
    with open(filename,'r') as f:
        for line in f.readlines():
            string += line.strip()
    return string

def convert_to_string(rootdir):
    file_list = os.listdir(rootdir)
    x = []
    for file in file_list:
        if file.endswith('.php'):
            file_path = os.path.join(rootdir,file)
            x.append(get_one_file(file_path))
    return x

（2）形成基于2-gram算法称重的全局词汇表。
这里仍然可以使用CountVectorizer用来向量化，但是需要特别说明3个参数：

- **ngram-range(min_n,max_n)：**提取的不同ngram的n值的上下界，设置成（2，2）表示基于2-gram
- **decode-errors{‘strict’, ‘ignore’, ‘replace’}：**设置成ignore，表明忽略异常字符的影响；strict将会引起报错。
- **token_pattern：**设置成r'\bw+b'，表明按照单词切割。

In [3]:
def webshell_vectorize():
    wb_vectorizer = CountVectorizer(ngram_range=(2,2),decode_error='ignore',token_pattern=r'\bw+\b',min_df=1)
    wb_file_list = convert_to_string("../data/PHP-WEBSHELL/dama/")
    x1 = wb_vectorizer.fit_transform(wb_file_list).toarray()
    vocabulary = wb_vectorizer.vocabulary_
    return x1,vocabulary

#### 2.特征化
使用webshell样本生成的词汇表vocabulary，将wordpress样本特征化。其中**最重要的是设置CountVectorizer函数的vocabulary，这样采用使用黑样本的词汇表进行向量化**。

In [4]:
def get_feature():
    x1,vocabulary = webshell_vectorize()
    wp_vectorizer = CountVectorizer(ngram_range=(2,2),decode_error='ignore',token_pattern=r'\bw+\b',min_df=1,vocabulary=vocabulary)
    wp_file_list = convert_to_string("../data/wordpress")
    x2 = wp_vectorizer.fit_transform(wp_file_list).toarray()

    y1,y2 = np.ones((x1.shape[0],1)),np.zeros((x2.shape[0],1))

    X = np.vstack((x1,x2))
    Y = np.vstack((y1,y2)).ravel()
    return X,Y

#### 3. 训练样本和效果验证

In [5]:
def main():
    X,Y = get_feature()
    clf = GaussianNB()
    score = cross_val_score(clf,X,Y,cv=3)

    print("score=",score)
main()

score= [ 0.90384615  0.86538462  0.76470588]


### 7.5 示例：检测WebShell（二）
#### 1.数据搜集和数据清洗
我们可以把WebShell看作为远程管理的一系列功能函数的调用，所以我们可以针对函数调用建立特征。针对黑样本集合，以1-gram算法形成全局的词汇表，切割方法为提取函数调用的名称。

In [6]:
def wbshell_vectorize_new():
    wbshell_vectorizer = CountVectorizer(ngram_range=(1,1),decode_error='ignore',token_pattern=r'\b\w+\b\(|\'\w+\'',min_df=1)
    wb_file_list = convert_to_string("../data/PHP-WEBSHELL/dama/")
    x1 = wbshell_vectorizer.fit_transform(wb_file_list).toarray()
    vocabulary = wbshell_vectorizer.vocabulary_
    return x1,vocabulary

#### 2.特征化

In [7]:
def get_feature_new():
    x1,vocabulary = wbshell_vectorize_new()
    wp_vectorizer = CountVectorizer(ngram_range=(1,1),decode_error='ignore',token_pattern=r'\b\w+\b\(|\'\w+\'',min_df=1,vocabulary=vocabulary)
    wp_file_list = convert_to_string("../data/wordpress")
    x2 = wp_vectorizer.fit_transform(wp_fi9le_list).toarray()
    y1, y2 = np.ones((x1.shape[0], 1)), np.zeros((x2.shape[0], 1))

    X = np.vstack((x1, x2))
    Y = np.vstack((y1, y2)).ravel()
    return X, Y

In [8]:
def main_1():
    X,Y = get_feature_new()
    clf = GaussianNB()
    score = cross_val_score(clf,X,Y,cv=3)

    print("score=",score)
main_1()

score= [ 0.71153846  0.96153846  0.70588235]
